In [1]:
import pandas as pd
import numpy as np

df_schedules = pd.read_pickle('../resources/data/preprocessed/schedules.pkl')

df_schedules

,season,day_number,backup_day_number,opponent,team_id,team,starter1,starter2,starter3,starter4,starter5,opponent_id
0,2011,7,6,UC Irvine,1228,Illinois,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale,1414.0
1,2011,7,6,Seattle,1268,Maryland,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams,1370.0
2,2011,7,6,Texas,1298,Navy,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey,1400.0
3,2011,7,6,Rhode Island,1338,Pittsburgh,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna,1348.0
4,2011,7,6,Pittsburgh,1348,Rhode Island,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond,1338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
134159,2023,148,147,Utah Valley,1412,UAB,T. Brewer,K. Buffen,T. Jemison,T. Lovan,J. Walker,1430.0
134160,2023,148,147,UAB,1430,Utah Valley,A. Bandaogo,L. Darthard,T. Fuller,J. Harmon,T. Woodbury,1412.0
134161,2023,148,147,North Texas,1458,Wisconsin,S. Crowl,C. Essegian,C. Hepburn,M. Klesmit,T. Wahl,1317.0
134162,2023,150,149,UAB,1317,North Texas,K. Huntsberry,R. Jones,T. Perry,A. Scott,M. Sissoko,1412.0


In [2]:
df_reg = pd.read_csv('../resources/data/unprocessed/MRegularSeasonCompactResults.csv')

df_reg = df_reg.loc[(df_reg['Season'] >= 2011) & (df_reg['Season'] != 2020), ].reset_index(drop=True)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2011,7,1228,79,1414,65,H,0
1,2011,7,1268,105,1370,76,H,0
2,2011,7,1338,83,1348,75,H,0
3,2011,7,1400,83,1298,52,H,0
4,2011,9,1228,84,1405,45,H,0
...,...,...,...,...,...,...,...,...
62964,2023,132,1104,82,1401,63,N,0
62965,2023,132,1272,75,1222,65,A,0
62966,2023,132,1343,74,1463,65,H,0
62967,2023,132,1345,67,1336,65,N,0


In [3]:
df_teams = pd.read_csv('../resources/data/unprocessed/MTeams.csv')

df_teams

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023
...,...,...,...,...
372,1473,Lindenwood,2023,2023
373,1474,Queens NC,2023,2023
374,1475,Southern Indiana,2023,2023
375,1476,Stonehill,2023,2023


In [4]:
df_reg['WTeamName'] = pd.merge(
    df_reg, df_teams, how='left', 
    left_on=['WTeamID'], right_on=['TeamID']
)['TeamName']

df_reg['LTeamName'] = pd.merge(
    df_reg, df_teams, how='left', 
    left_on=['LTeamID'], right_on=['TeamID']
)['TeamName']

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island
3,2011,7,1400,83,1298,52,H,0,Texas,Navy
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo
...,...,...,...,...,...,...,...,...,...,...
62964,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M
62965,2023,132,1272,75,1222,65,A,0,Memphis,Houston
62966,2023,132,1343,74,1463,65,H,0,Princeton,Yale
62967,2023,132,1345,67,1336,65,N,0,Purdue,Penn St


In [5]:
df_reg_reshape = pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'team_id': df_reg['WTeamID'], 
                               'team_name': df_reg['WTeamName'], 
                               'opponent_id': df_reg['LTeamID'], 
                               'opponent_name': df_reg['LTeamName'], 
                               'result': np.repeat('win', df_reg.shape[0])})

temp =           pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'team_id': df_reg['LTeamID'], 
                               'team_name': df_reg['LTeamName'], 
                               'opponent_id': df_reg['WTeamID'], 
                               'opponent_name': df_reg['WTeamName'], 
                               'result': np.repeat('loss', df_reg.shape[0])})

df_reg_reshape = pd.concat([df_reg_reshape, temp], axis=0).reset_index(drop=True)

df_reg_reshape

,season,day_number,team_id,team_name,opponent_id,opponent_name,result
0,2011,7,1228,Illinois,1414,UC Irvine,win
1,2011,7,1268,Maryland,1370,Seattle,win
2,2011,7,1338,Pittsburgh,1348,Rhode Island,win
3,2011,7,1400,Texas,1298,Navy,win
4,2011,9,1228,Illinois,1405,Toledo,win
...,...,...,...,...,...,...,...
125933,2023,132,1401,Texas A&M,1104,Alabama,loss
125934,2023,132,1222,Houston,1272,Memphis,loss
125935,2023,132,1463,Yale,1343,Princeton,loss
125936,2023,132,1336,Penn St,1345,Purdue,loss


In [6]:
temp1 = df_reg_reshape.merge(
    df_schedules, how='inner', on=['season', 'day_number', 'team_id', 'opponent_id']
).drop(columns=['backup_day_number', 'opponent', 'team'])

temp1

,season,day_number,team_id,team_name,opponent_id,opponent_name,result,starter1,starter2,starter3,starter4,starter5
0,2011,7,1228,Illinois,1414,UC Irvine,win,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,1268,Maryland,1370,Seattle,win,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,1338,Pittsburgh,1348,Rhode Island,win,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
3,2011,7,1400,Texas,1298,Navy,win,D. Balbay,J. Hamilton,M. Hill,G. Johnson,C. Joseph
4,2011,9,1228,Illinois,1405,Toledo,win,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
...,...,...,...,...,...,...,...,...,...,...,...,...
124634,2023,132,1401,Texas A&M,1104,Alabama,loss,H. Coleman,D. Dennis,J. Marble,T. Radford,W. Taylor
124635,2023,132,1222,Houston,1272,Memphis,loss,T. Arceneaux,T. Mark,J. Roberts,J. Shead,J. Walker
124636,2023,132,1463,Yale,1343,Princeton,loss,E. Jarvis,I. Kelly,A. Mahoney,B. Mbeng,J. Poulakidas
124637,2023,132,1336,Penn St,1345,Purdue,loss,A. Funk,S. Lundy,K. Njie,J. Pickett,C. Wynter


In [7]:
temp1.columns

Index(['season', 'day_number', 'team_id', 'team_name', 'opponent_id',
       'opponent_name', 'result', 'starter1', 'starter2', 'starter3',
       'starter4', 'starter5'],
      dtype='object')

In [8]:
# in case day numbers got desynced

temp2 = df_reg_reshape.merge(
    df_schedules, how='inner', 
    left_on=['season', 'day_number', 'team_id', 'opponent_id'], 
    right_on=['season', 'backup_day_number', 'team_id', 'opponent_id'], 
    suffixes = ('', '_y')
).drop(columns=['day_number_y', 'backup_day_number', 'opponent', 'team'])

temp2

,season,day_number,team_id,team_name,opponent_id,opponent_name,result,starter1,starter2,starter3,starter4,starter5
0,2011,12,1286,Montana St,1141,C Michigan,win,C. Anderson,B. Howard,D. Piepoli,E. Rush,R. Singleton
1,2011,14,1218,Hawaii,1141,C Michigan,win,B. Amis,V. Joaquim,Z. Johnson,J. Thomas,H. Thompson
2,2011,14,1388,St Mary's CA,1385,St John's,win,S. Holt,R. Jones,M. McConnell,C. Steindl,K. Walker
3,2011,18,1218,Hawaii,1146,Cent Arkansas,win,B. Amis,V. Joaquim,Z. Johnson,J. Thomas,H. Thompson
4,2011,23,1385,St John's,1123,Ball St,win,M. Boothe,J. Brownlee,D. Hardy,D. Kennedy,D. Polee
...,...,...,...,...,...,...,...,...,...,...,...,...
1497,2023,101,1471,UC San Diego,1218,Hawaii,loss,R. Anderson,J. Kosakowski,F. Nwaokorie,B. Pope,J. Roquemore
1498,2023,103,1218,Hawaii,1168,CS Fullerton,loss,S. Avea,N. Coleman,B. DaSilva,K. Hepa,J. McClanahan
1499,2023,115,1218,Hawaii,1415,UC Riverside,loss,S. Avea,N. Coleman,B. DaSilva,K. Hepa,J. McClanahan
1500,2023,124,1365,Santa Clara,1362,San Francisco,loss,J. Bediako,P. Braun,K. Justice,B. Podziemski,C. Stewart


In [9]:
df_reg_reshape = pd.concat(
    [temp1, temp2], ignore_index=True
).sort_values(['season', 'day_number', 'team_id'], ignore_index=True)

df_reg_reshape

,season,day_number,team_id,team_name,opponent_id,opponent_name,result,starter1,starter2,starter3,starter4,starter5
0,2011,7,1228,Illinois,1414,UC Irvine,win,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,1268,Maryland,1370,Seattle,win,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,1298,Navy,1400,Texas,loss,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey
3,2011,7,1338,Pittsburgh,1348,Rhode Island,win,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
4,2011,7,1348,Rhode Island,1338,Pittsburgh,loss,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond
...,...,...,...,...,...,...,...,...,...,...,...,...
126136,2023,132,1343,Princeton,1463,Yale,win,M. Allocco,T. Evbuomwan,K. Kellman,R. Langborg,C. Pierce
126137,2023,132,1345,Purdue,1336,Penn St,win,Z. Edey,M. Gillis,F. Loyer,B. Newman,B. Smith
126138,2023,132,1401,Texas A&M,1104,Alabama,loss,H. Coleman,D. Dennis,J. Marble,T. Radford,W. Taylor
126139,2023,132,1433,VCU,1173,Dayton,win,A. Baldwin,J. Deloach,B. Johns,N. Kern,J. Nunn


In [10]:
df_reg = df_reg.merge(
    df_reg_reshape, how='inner', 
    left_on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], 
    right_on=['season', 'day_number', 'team_id', 'opponent_id']
).drop(
    columns=['season', 'day_number', 'team_id', 
             'team_name', 'opponent_id', 'opponent_name', 'result']
).rename(
    columns={
        'starter1': 'WTeamStarter1', 
        'starter2': 'WTeamStarter2', 
        'starter3': 'WTeamStarter3', 
        'starter4': 'WTeamStarter4', 
        'starter5': 'WTeamStarter5'
    }
)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,D. Balbay,J. Hamilton,M. Hill,G. Johnson,C. Joseph
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62956,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,C. Bediako,N. Clowney,B. Miller,J. Quinerly,M. Sears
62957,2023,132,1272,75,1222,65,A,0,Memphis,Houston,K. Davis,J. Hardaway,C. Lawson,E. McCadden,D. Williams
62958,2023,132,1343,74,1463,65,H,0,Princeton,Yale,M. Allocco,T. Evbuomwan,K. Kellman,R. Langborg,C. Pierce
62959,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,Z. Edey,M. Gillis,F. Loyer,B. Newman,B. Smith


In [11]:
df_reg = df_reg.merge(
    df_reg_reshape, how='inner', 
    left_on=['Season', 'DayNum', 'LTeamID', 'WTeamID'], 
    right_on=['season', 'day_number', 'team_id', 'opponent_id']
).drop(
    columns=['season', 'day_number', 'team_id', 
             'team_name', 'opponent_id', 'opponent_name', 'result']
).rename(
    columns={
        'starter1': 'LTeamStarter1', 
        'starter2': 'LTeamStarter2', 
        'starter3': 'LTeamStarter3', 
        'starter4': 'LTeamStarter4', 
        'starter5': 'LTeamStarter5'
    }
)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale,P. Losonsky,D. Moore,P. Rembert,D. Starring,E. Wise
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams,A. Broussard,C. Burrell,S. Carter,G. Gilmore,G. Lever
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,D. Balbay,J. Hamilton,M. Hill,G. Johnson,C. Joseph,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale,J. Anyijong,D. Dear,M. Griffin,R. Holliday,Z. Leahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63814,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,C. Bediako,N. Clowney,B. Miller,J. Quinerly,M. Sears,H. Coleman,D. Dennis,J. Marble,T. Radford,W. Taylor
63815,2023,132,1272,75,1222,65,A,0,Memphis,Houston,K. Davis,J. Hardaway,C. Lawson,E. McCadden,D. Williams,T. Arceneaux,T. Mark,J. Roberts,J. Shead,J. Walker
63816,2023,132,1343,74,1463,65,H,0,Princeton,Yale,M. Allocco,T. Evbuomwan,K. Kellman,R. Langborg,C. Pierce,E. Jarvis,I. Kelly,A. Mahoney,B. Mbeng,J. Poulakidas
63817,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,Z. Edey,M. Gillis,F. Loyer,B. Newman,B. Smith,A. Funk,S. Lundy,K. Njie,J. Pickett,C. Wynter


In [12]:
for i in range(1, 6):
    df_reg[f'WTeamStarter{i}'] = df_reg['WTeamID'].astype(str) + ' ' + df_reg[f'WTeamStarter{i}']
    df_reg[f'LTeamStarter{i}'] = df_reg['LTeamID'].astype(str) + ' ' + df_reg[f'LTeamStarter{i}']
    
df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,WTeamStarter1,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,1268 A. Bowie,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,1338 G. Brown,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,1400 D. Balbay,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63814,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,1104 C. Bediako,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor
63815,2023,132,1272,75,1222,65,A,0,Memphis,Houston,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker
63816,2023,132,1343,74,1463,65,H,0,Princeton,Yale,1343 M. Allocco,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas
63817,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,1345 Z. Edey,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter


In [13]:
df_reg['ScoreDiff'] = df_reg['WScore'] - df_reg['LScore']

# df_reg.loc[df_reg['ScoreDiff'] > 15, 'ScoreDiff'] = 15

df_reg.loc[df_reg['NumOT'] > 0, 'ScoreDiff'] = 1

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,...,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5,ScoreDiff
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,...,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise,14
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,...,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever,29
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,...,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond,8
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,...,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey,31
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,...,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63814,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,...,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,19
63815,2023,132,1272,75,1222,65,A,0,Memphis,Houston,...,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,10
63816,2023,132,1343,74,1463,65,H,0,Princeton,Yale,...,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,9
63817,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,...,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,2


Flip data for symmetry

In [14]:
df_reg_reshape = pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'result': np.repeat(0, df_reg.shape[0]), 
                               'score_diff': df_reg['ScoreDiff'], 
                               'team0': df_reg['WTeamID'], 
                               'team0_name': df_reg['WTeamName'], 
                               'team0_loc': df_reg['WLoc'], 
                               'team1': df_reg['LTeamID'], 
                               'team1_name': df_reg['LTeamName'], 
                               'team0_starter1': df_reg['WTeamStarter1'], 
                               'team0_starter2': df_reg['WTeamStarter2'], 
                               'team0_starter3': df_reg['WTeamStarter3'], 
                               'team0_starter4': df_reg['WTeamStarter4'], 
                               'team0_starter5': df_reg['WTeamStarter5'], 
                               'team1_starter1': df_reg['LTeamStarter1'], 
                               'team1_starter2': df_reg['LTeamStarter2'], 
                               'team1_starter3': df_reg['LTeamStarter3'], 
                               'team1_starter4': df_reg['LTeamStarter4'], 
                               'team1_starter5': df_reg['LTeamStarter5']})

temp =           pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'result': np.repeat(1, df_reg.shape[0]), 
                               'score_diff': -df_reg['ScoreDiff'], 
                               'team0': df_reg['LTeamID'], 
                               'team0_name': df_reg['LTeamName'], 
                               'team0_loc': df_reg['WLoc'].map({'H': 'A', 'A': 'H', 'N': 'N'}), 
                               'team1': df_reg['WTeamID'], 
                               'team1_name': df_reg['WTeamName'], 
                               'team0_starter1': df_reg['LTeamStarter1'], 
                               'team0_starter2': df_reg['LTeamStarter2'], 
                               'team0_starter3': df_reg['LTeamStarter3'], 
                               'team0_starter4': df_reg['LTeamStarter4'], 
                               'team0_starter5': df_reg['LTeamStarter5'], 
                               'team1_starter1': df_reg['WTeamStarter1'], 
                               'team1_starter2': df_reg['WTeamStarter2'], 
                               'team1_starter3': df_reg['WTeamStarter3'], 
                               'team1_starter4': df_reg['WTeamStarter4'], 
                               'team1_starter5': df_reg['WTeamStarter5']})

df_reg_reshape = pd.concat([df_reg_reshape, temp], axis=0).reset_index(drop=True)

df_reg_reshape

,season,day_number,result,score_diff,team0,team0_name,team0_loc,team1,team1_name,team0_starter1,team0_starter2,team0_starter3,team0_starter4,team0_starter5,team1_starter1,team1_starter2,team1_starter3,team1_starter4,team1_starter5
0,2011,7,0,14,1228,Illinois,H,1414,UC Irvine,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise
1,2011,7,0,29,1268,Maryland,H,1370,Seattle,1268 A. Bowie,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever
2,2011,7,0,8,1338,Pittsburgh,H,1348,Rhode Island,1338 G. Brown,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond
3,2011,7,0,31,1400,Texas,H,1298,Navy,1400 D. Balbay,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey
4,2011,9,0,39,1228,Illinois,H,1405,Toledo,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127633,2023,132,1,-19,1401,Texas A&M,N,1104,Alabama,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,1104 C. Bediako,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears
127634,2023,132,1,-10,1222,Houston,H,1272,Memphis,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams
127635,2023,132,1,-9,1463,Yale,A,1343,Princeton,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,1343 M. Allocco,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce
127636,2023,132,1,-2,1336,Penn St,N,1345,Purdue,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,1345 Z. Edey,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith


Determine home court advantage

In [15]:
df_reg_reshape_home = df_reg_reshape.loc[df_reg_reshape['team0_loc'] == 'H', 
                                         ['season', 'score_diff', 'team0', 'team1']].copy()

df_reg_reshape_home

,season,score_diff,team0,team1
0,2011,14,1228,1414
1,2011,29,1268,1370
2,2011,8,1338,1348
3,2011,31,1400,1298
4,2011,39,1228,1405
...,...,...,...,...
127418,2023,-2,1272,1222
127425,2023,-4,1234,1304
127428,2023,-12,1353,1321
127439,2023,-4,1278,1458


In [16]:
df_reg_reshape_neutral = df_reg_reshape.loc[df_reg_reshape['team0_loc'] == 'N', 
                                            ['season', 'score_diff', 'team0', 'team1']].copy()

df_reg_reshape_neutral

,season,score_diff,team0,team1
17,2011,3,1141,1168
26,2011,7,1194,1413
38,2011,23,1232,1410
42,2011,1,1245,1233
79,2011,16,1364,1176
...,...,...,...,...
127630,2023,-12,1387,1433
127632,2023,-20,1165,1463
127633,2023,-19,1401,1104
127636,2023,-2,1336,1345


In [17]:
df_home_court_advantage = pd.merge(
    df_reg_reshape_home, df_reg_reshape_neutral, how='inner', 
    on=['season', 'team0', 'team1'], suffixes=('_home', '_neutral')
)

df_home_court_advantage['advantage'] = (
    df_home_court_advantage['score_diff_home'] - df_home_court_advantage['score_diff_neutral']
)

df_home_court_advantage

,season,score_diff_home,team0,team1,score_diff_neutral,advantage
0,2011,26,1331,1114,-3,29
1,2011,1,1187,1150,-11,12
2,2011,16,1324,1381,16,0
3,2011,8,1265,1310,12,-4
4,2011,8,1174,1330,-9,17
...,...,...,...,...,...,...
5123,2023,-5,1431,1443,-6,1
5124,2023,-3,1380,1105,-14,11
5125,2023,-2,1350,1206,-5,3
5126,2023,-15,1124,1235,-6,-9


In [18]:
df_home_court_advantage['advantage'].mean()

3.468603744149766

In [19]:
df_home_court_advantage['advantage'].median()

3.0

In [20]:
df_reg_reshape_away = df_reg_reshape.loc[df_reg_reshape['team0_loc'] == 'A', 
                                         ['season', 'score_diff', 'team0', 'team1']].copy()

df_reg_reshape_away

,season,score_diff,team0,team1
10,2011,3,1111,1409
11,2011,10,1114,1374
12,2011,2,1122,1387
15,2011,4,1135,1200
23,2011,4,1165,1107
...,...,...,...,...
127616,2023,-21,1153,1222
127617,2023,-1,1313,1224
127623,2023,-7,1335,1343
127631,2023,-13,1262,1436


In [21]:
df_home_court_advantage = pd.merge(
    df_reg_reshape_home, df_reg_reshape_away, how='inner', 
    on=['season', 'team0', 'team1'], suffixes=('_home', '_away')
)

df_home_court_advantage['advantage'] = (
    (df_home_court_advantage['score_diff_home'] - df_home_court_advantage['score_diff_away'])/2
)

df_home_court_advantage['advantage'].mean()

2.947464682139253

In [22]:
df_home_court_advantage['advantage'].median()

3.0

3 points seems reasonable

In [23]:
df_reg.loc[df_reg['WLoc'] == 'H', 'ScoreDiff'] = df_reg.loc[df_reg['WLoc'] == 'H', 'ScoreDiff'] - 3
df_reg.loc[df_reg['WLoc'] == 'A', 'ScoreDiff'] = df_reg.loc[df_reg['WLoc'] == 'A', 'ScoreDiff'] + 3

df_reg.loc[df_reg['ScoreDiff'] < 1, 'ScoreDiff'] = 1

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,...,WTeamStarter2,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5,ScoreDiff
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,...,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise,11
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,...,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever,26
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,...,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond,5
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,...,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey,28
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,...,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63814,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,...,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,19
63815,2023,132,1272,75,1222,65,A,0,Memphis,Houston,...,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,13
63816,2023,132,1343,74,1463,65,H,0,Princeton,Yale,...,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,6
63817,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,...,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,2


Take log of score because a 1 point win vs 2 point win should be more important than 19 points vs 20 points, for example

In [24]:
df_reg['LogScoreDiff'] = np.log(df_reg['ScoreDiff'])

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName,...,WTeamStarter3,WTeamStarter4,WTeamStarter5,LTeamStarter1,LTeamStarter2,LTeamStarter3,LTeamStarter4,LTeamStarter5,ScoreDiff,LogScoreDiff
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine,...,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise,11,2.397895
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle,...,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever,26,3.258097
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island,...,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond,5,1.609438
3,2011,7,1400,83,1298,52,H,0,Texas,Navy,...,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey,28,3.332205
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo,...,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy,36,3.583519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63814,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M,...,1104 B. Miller,1104 J. Quinerly,1104 M. Sears,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,19,2.944439
63815,2023,132,1272,75,1222,65,A,0,Memphis,Houston,...,1272 C. Lawson,1272 E. McCadden,1272 D. Williams,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,13,2.564949
63816,2023,132,1343,74,1463,65,H,0,Princeton,Yale,...,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,6,1.791759
63817,2023,132,1345,67,1336,65,N,0,Purdue,Penn St,...,1345 F. Loyer,1345 B. Newman,1345 B. Smith,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,2,0.693147


In [25]:
df_reg_reshape = pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'result': np.repeat(0, df_reg.shape[0]), 
                               'log_score_diff': df_reg['LogScoreDiff'], 
                               'team0': df_reg['WTeamID'], 
                               'team0_name': df_reg['WTeamName'], 
                               'team0_loc': df_reg['WLoc'], 
                               'team1': df_reg['LTeamID'], 
                               'team1_name': df_reg['LTeamName'], 
                               'team0_starter1': df_reg['WTeamStarter1'], 
                               'team0_starter2': df_reg['WTeamStarter2'], 
                               'team0_starter3': df_reg['WTeamStarter3'], 
                               'team0_starter4': df_reg['WTeamStarter4'], 
                               'team0_starter5': df_reg['WTeamStarter5'], 
                               'team1_starter1': df_reg['LTeamStarter1'], 
                               'team1_starter2': df_reg['LTeamStarter2'], 
                               'team1_starter3': df_reg['LTeamStarter3'], 
                               'team1_starter4': df_reg['LTeamStarter4'], 
                               'team1_starter5': df_reg['LTeamStarter5']})

temp =           pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'result': np.repeat(1, df_reg.shape[0]), 
                               'log_score_diff': -df_reg['LogScoreDiff'], 
                               'team0': df_reg['LTeamID'], 
                               'team0_name': df_reg['LTeamName'], 
                               'team0_loc': df_reg['WLoc'].map({'H': 'A', 'A': 'H', 'N': 'N'}), 
                               'team1': df_reg['WTeamID'], 
                               'team1_name': df_reg['WTeamName'], 
                               'team0_starter1': df_reg['LTeamStarter1'], 
                               'team0_starter2': df_reg['LTeamStarter2'], 
                               'team0_starter3': df_reg['LTeamStarter3'], 
                               'team0_starter4': df_reg['LTeamStarter4'], 
                               'team0_starter5': df_reg['LTeamStarter5'], 
                               'team1_starter1': df_reg['WTeamStarter1'], 
                               'team1_starter2': df_reg['WTeamStarter2'], 
                               'team1_starter3': df_reg['WTeamStarter3'], 
                               'team1_starter4': df_reg['WTeamStarter4'], 
                               'team1_starter5': df_reg['WTeamStarter5']})

df_reg_reshape = pd.concat([df_reg_reshape, temp], axis=0).reset_index(drop=True)

df_reg_reshape

,season,day_number,result,log_score_diff,team0,team0_name,team0_loc,team1,team1_name,team0_starter1,team0_starter2,team0_starter3,team0_starter4,team0_starter5,team1_starter1,team1_starter2,team1_starter3,team1_starter4,team1_starter5
0,2011,7,0,2.397895,1228,Illinois,H,1414,UC Irvine,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise
1,2011,7,0,3.258097,1268,Maryland,H,1370,Seattle,1268 A. Bowie,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever
2,2011,7,0,1.609438,1338,Pittsburgh,H,1348,Rhode Island,1338 G. Brown,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond
3,2011,7,0,3.332205,1400,Texas,H,1298,Navy,1400 D. Balbay,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey
4,2011,9,0,3.583519,1228,Illinois,H,1405,Toledo,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127633,2023,132,1,-2.944439,1401,Texas A&M,N,1104,Alabama,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,1104 C. Bediako,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears
127634,2023,132,1,-2.564949,1222,Houston,H,1272,Memphis,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams
127635,2023,132,1,-1.791759,1463,Yale,A,1343,Princeton,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,1343 M. Allocco,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce
127636,2023,132,1,-0.693147,1336,Penn St,N,1345,Purdue,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,1345 Z. Edey,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith


In [26]:
df_mod = df_reg_reshape.loc[
    df_reg_reshape['season'] == 2023, 
    ['result', 'log_score_diff'] + list(df_reg_reshape.filter(regex='starter').columns)].reset_index(
    drop=True
).copy()

df_mod

,result,log_score_diff,team0_starter1,team0_starter2,team0_starter3,team0_starter4,team0_starter5,team1_starter1,team1_starter2,team1_starter3,team1_starter4,team1_starter5
0,0,1.791759,1101 I. Allen,1101 T. Cameron,1101 D. Daniels,1101 A. Simmons,1101 C. Steele,1238 C. Adams,1238 K. Evans,1238 R. Mansel,1238 G. Watson,1238 T. Young
1,0,0.000000,1103 X. Castaneda,1103 E. Freeman,1103 T. Hankerson,1103 S. Hunter,1103 G. Tribble,1355 A. Arians,1355 M. Dentlinger,1355 C. Easley,1355 W. Kyle,1355 Z. Mayo
2,0,2.890372,1104 C. Bediako,1104 N. Burnett,1104 N. Clowney,1104 B. Miller,1104 M. Sears,1255 M. Christmas,1255 D. Houston,1255 L. Nkereuwem,1255 D. Wade,1255 I. Wilkins
3,0,3.663562,1112 O. Ballo,1112 C. Henderson,1112 K. Kriisa,1112 P. Larsson,1112 A. Tubelis,1311 C. Huffman,1311 L. Jones,1311 M. Nelson,1311 L. Paul,1311 P. Spencer
4,0,0.000000,1113 M. Bagley,1113 D. Cambridge,1113 F. Collins,1113 D. Horne,1113 W. Washington,1470 S. Bogues,1470 S. Daniel,1470 M. Gatkuoth,1470 N. McDavid,1470 J. Smith
...,...,...,...,...,...,...,...,...,...,...,...,...
11089,1,-2.944439,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,1104 C. Bediako,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears
11090,1,-2.564949,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams
11091,1,-1.791759,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,1343 M. Allocco,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce
11092,1,-0.693147,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,1345 Z. Edey,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith


In [27]:
temp = pd.get_dummies(df_mod.drop(columns=['result', 'log_score_diff']))

y_train = df_mod['result'].copy()

temp

,team0_starter1_1101 I. Allen,team0_starter1_1101 T. Cameron,team0_starter1_1102 B. Becker,team0_starter1_1102 C. Green,team0_starter1_1102 J. Heidbreder,team0_starter1_1103 E. Freeman,team0_starter1_1103 X. Castaneda,team0_starter1_1104 C. Bediako,team0_starter1_1105 A. Harvell,team0_starter1_1105 G. Hicks,...,team1_starter5_1473 B. Trimble,team1_starter5_1473 D. Ware,team1_starter5_1474 B. McLaurin,team1_starter5_1474 G. Rains,team1_starter5_1474 J. Turner,team1_starter5_1475 G. Solomon,team1_starter5_1475 I. Swope,team1_starter5_1476 M. Zegarowski,team1_starter5_1477 K. Williams,team1_starter5_1477 L. Vasic
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11092,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
temp.columns = temp.columns.str[:5] + temp.columns.str[14:]

temp.columns

Index(['team0_1101 I. Allen', 'team0_1101 T. Cameron', 'team0_1102 B. Becker',
       'team0_1102 C. Green', 'team0_1102 J. Heidbreder',
       'team0_1103 E. Freeman', 'team0_1103 X. Castaneda',
       'team0_1104 C. Bediako', 'team0_1105 A. Harvell', 'team0_1105 G. Hicks',
       ...
       'team1_1473 B. Trimble', 'team1_1473 D. Ware', 'team1_1474 B. McLaurin',
       'team1_1474 G. Rains', 'team1_1474 J. Turner', 'team1_1475 G. Solomon',
       'team1_1475 I. Swope', 'team1_1476 M. Zegarowski',
       'team1_1477 K. Williams', 'team1_1477 L. Vasic'],
      dtype='object', length=10588)

In [29]:
X_train = temp.groupby(temp.columns, axis=1).sum()

X_train

,team0_1101 A. Dibba,team0_1101 A. Simmons,team0_1101 C. Steele,team0_1101 D. Daniels,team0_1101 H. Madden,team0_1101 I. Allen,team0_1101 J. Jackson,team0_1101 J. Pleasant,team0_1101 T. Cameron,team0_1102 B. Becker,...,team1_1477 A. Peavy,team1_1477 C. Roberts,team1_1477 D. Demonia,team1_1477 J. Brewer,team1_1477 J. RomerRosario,team1_1477 K. Abdul-Mateen,team1_1477 K. Williams,team1_1477 L. Vasic,team1_1477 M. Davis,team1_1477 T. Lewis
0,0,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11089,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11090,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11091,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11092,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Some players are labeled the same because of names :( 

For now, just treat as a single player with 2x impact

In [30]:
X_train.iloc[41:42, 4411:4412]

,team1_1237 A. Jarrard
41,2


In [31]:
from sklearn.linear_model import LogisticRegression

mod = LogisticRegression(penalty='l2', max_iter=1000, fit_intercept=False)

mod.fit(X_train, y_train)

'Done'

'Done'

In [32]:
temp = pd.DataFrame(
    {'Player': X_train.columns, 'Rating': mod.coef_.round(2)[0, :]}
).sort_values(['Rating'], ascending=False, ignore_index=True)

temp = temp.loc[temp['Player'].str.contains('team1'), :].reset_index(drop=True)

temp

,Player,Rating
0,team1_1345 M. Gillis,1.29
1,team1_1439 H. Cattoor,1.27
2,team1_1129 N. Smith,1.26
3,team1_1113 D. Horne,1.21
4,team1_1403 K. Walton,1.20
...,...,...
3231,team1_1291 F. Barton,-1.21
3232,team1_1144 J. Lusane,-1.26
3233,team1_1269 K. Thompson,-1.31
3234,team1_1408 C. Holloway,-1.39


In [33]:
temp.head(20)

,Player,Rating
0,team1_1345 M. Gillis,1.29
1,team1_1439 H. Cattoor,1.27
2,team1_1129 N. Smith,1.26
3,team1_1113 D. Horne,1.21
4,team1_1403 K. Walton,1.20
5,team1_1307 J. House,1.19
6,team1_1116 R. Council,1.16
7,team1_1395 M. Miles,1.13
8,team1_1194 B. Greenlee,1.12
9,team1_1274 N. Pack,1.12


In [34]:
temp.loc[temp['Player'].str.contains('J. Clark'), :]

,Player,Rating
786,team1_1417 J. Clark,0.29
1729,team1_1182 J. Clark,-0.04
1844,team1_1301 J. Clark,-0.08


In [35]:
temp.loc[temp['Player'].str.contains('1104'), :]

,Player,Rating
54,team1_1104 M. Sears,0.86
55,team1_1104 B. Miller,0.86
56,team1_1104 C. Bediako,0.86
70,team1_1104 N. Clowney,0.81
202,team1_1104 J. Quinerly,0.63
469,team1_1104 N. Burnett,0.43
1449,team1_1104 N. Gurley,0.04
2240,team1_1104 J. Bradley,-0.20


In [36]:
y_train = df_mod['log_score_diff'].copy()

y_train

0        1.791759
1        0.000000
2        2.890372
3        3.663562
4        0.000000
           ...   
11089   -2.944439
11090   -2.564949
11091   -1.791759
11092   -0.693147
11093   -2.484907
Name: log_score_diff, Length: 11094, dtype: float64

In [37]:
from sklearn.linear_model import Ridge

mod = Ridge(fit_intercept=False)

mod.fit(X_train, y_train)

'Done'

'Done'

In [38]:
temp = pd.DataFrame(
    {'Player': X_train.columns, 'Rating': mod.coef_.round(2)}
).sort_values(['Rating'], ascending=False, ignore_index=True)

temp = temp.loc[temp['Player'].str.contains('team0'), :].reset_index(drop=True)

temp

,Player,Rating
0,team0_1426 M. Young,2.19
1,team0_1370 A. Schumacher,2.00
2,team0_1307 J. House,1.93
3,team0_1185 N. Farrakhan,1.77
4,team0_1101 J. Pleasant,1.73
...,...,...
3231,team0_1282 E. Dimou,-1.95
3232,team0_1269 K. Thompson,-2.04
3233,team0_1224 O. Okojie,-2.05
3234,team0_1433 T. Lawal,-2.23


In [39]:
temp.head(20)

,Player,Rating
0,team0_1426 M. Young,2.19
1,team0_1370 A. Schumacher,2.00
2,team0_1307 J. House,1.93
3,team0_1185 N. Farrakhan,1.77
4,team0_1101 J. Pleasant,1.73
5,team0_1322 E. McDonald,1.69
6,team0_1390 H. Ingram,1.68
7,team0_1129 N. Smith,1.68
8,team0_1435 T. Lawrence,1.67
9,team0_1235 C. Grill,1.66


In [40]:
temp.loc[temp['Player'].str.contains('J. Clark'), :]

,Player,Rating
1819,team0_1182 J. Clark,-0.08
2099,team0_1301 J. Clark,-0.21
2407,team0_1417 J. Clark,-0.37


In [41]:
temp.loc[temp['Player'].str.contains('1163'), :]

,Player,Rating
55,team0_1163 J. Hawkins,1.29
89,team0_1163 A. Karaban,1.15
157,team0_1163 N. Alleyne,1.00
293,team0_1163 A. Sanogo,0.77
905,team0_1163 H. Diarra,0.35
1885,team0_1163 T. Newton,-0.11
2174,team0_1163 A. Jackson,-0.24
2442,team0_1163 S. Johnson,-0.39


In [42]:
temp.loc[temp['Player'].str.contains('1345'), :]

,Player,Rating
17,team0_1345 Z. Edey,1.57
398,team0_1345 B. Smith,0.65
401,team0_1345 F. Loyer,0.65
447,team0_1345 M. Gillis,0.61
611,team0_1345 E. Morton,0.51
1310,team0_1345 B. Newman,0.14
3008,team0_1345 C. Furst,-0.89


Get team ratings

In [43]:
%%time

ratings = []

for season in df_reg_reshape['season'].unique():
    print(season)
    
    X_train = pd.get_dummies(
        df_reg_reshape.loc[df_reg_reshape['season'] == season, 
                           ['team0', 'team1']
                          ].reset_index(drop=True).copy(), 
        columns=['team0', 'team1']
    )
    
    y_train = df_reg_reshape.loc[df_reg_reshape['season'] == season, 
                                 'log_score_diff'
                                ].reset_index(drop=True).copy()
    
    
    mod = Ridge(fit_intercept=False)
    mod.fit(X_train, y_train)
    
    r = pd.DataFrame(
        {'Season': season, 'Team': X_train.columns, 'Rating': mod.coef_}
    )
    
    r = r.loc[r['Team'].str.contains('team0'), :].reset_index(drop=True)
    
    r['Team'] = r['Team'].str[6:]
    
    ratings.append(r)
    
df_team_ratings = pd.concat(
    ratings, ignore_index=True
).sort_values(
    ['Rating'], ascending=False, ignore_index=True
)

df_team_ratings['Team'] = df_team_ratings['Team'].astype(int)

df_team_ratings

2011
2012
2013
2014
2015
2016
2017
2018
2019
2021
2022
2023
CPU times: total: 14.4 s
Wall time: 2.52 s


,Season,Team,Rating
0,2015,1246,3.642318
1,2019,1181,3.301051
2,2021,1211,3.295913
3,2019,1277,3.272123
4,2011,1326,3.238398
...,...,...,...
4183,2023,1254,-3.208920
4184,2015,1197,-3.222542
4185,2021,1290,-3.333246
4186,2015,1212,-3.445263


In [44]:
temp1 = np.array(
    pd.merge(
        df_reg_reshape, df_team_ratings, 
        how='left', 
        left_on=['season', 'team1'], 
        right_on=['Season', 'Team']
    )['Rating']
)


temp0 = np.array(
    pd.merge(
        df_reg_reshape, df_team_ratings, 
        how='left', 
        left_on=['season', 'team0'], 
        right_on=['Season', 'Team']
    )['Rating']
)

df_reg_reshape['rating_diff'] = temp1 - temp0

df_reg_reshape

,season,day_number,result,log_score_diff,team0,team0_name,team0_loc,team1,team1_name,team0_starter1,team0_starter2,team0_starter3,team0_starter4,team0_starter5,team1_starter1,team1_starter2,team1_starter3,team1_starter4,team1_starter5,rating_diff
0,2011,7,0,2.397895,1228,Illinois,H,1414,UC Irvine,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1414 P. Losonsky,1414 D. Moore,1414 P. Rembert,1414 D. Starring,1414 E. Wise,-2.883339
1,2011,7,0,3.258097,1268,Maryland,H,1370,Seattle,1268 A. Bowie,1268 D. Gregory,1268 S. Mosley,1268 C. Tucker,1268 J. Williams,1370 A. Broussard,1370 C. Burrell,1370 S. Carter,1370 G. Gilmore,1370 G. Lever,-2.665005
2,2011,7,0,1.609438,1338,Pittsburgh,H,1348,Rhode Island,1338 G. Brown,1338 A. Gibbs,1338 G. McGhee,1338 B. Wanamaker,1338 T. Zanna,1348 R. Brooks,1348 D. James,1348 M. Jones,1348 W. Martell,1348 A. Richmond,-2.370071
3,2011,7,0,3.332205,1400,Texas,H,1298,Navy,1400 D. Balbay,1400 J. Hamilton,1400 M. Hill,1400 G. Johnson,1400 C. Joseph,1298 J. Avila,1298 O. Avworo,1298 R. Garcia,1298 J. Sugars,1298 M. Veazey,-3.795273
4,2011,9,0,3.583519,1228,Illinois,H,1405,Toledo,1228 B. Cole,1228 M. Davis,1228 D. McCamey,1228 D. Richardson,1228 M. Tisdale,1405 J. Anyijong,1405 D. Dear,1405 M. Griffin,1405 R. Holliday,1405 Z. Leahy,-4.270462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127633,2023,132,1,-2.944439,1401,Texas A&M,N,1104,Alabama,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,1104 C. Bediako,1104 N. Clowney,1104 B. Miller,1104 J. Quinerly,1104 M. Sears,0.968529
127634,2023,132,1,-2.564949,1222,Houston,H,1272,Memphis,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,1272 K. Davis,1272 J. Hardaway,1272 C. Lawson,1272 E. McCadden,1272 D. Williams,-0.692310
127635,2023,132,1,-1.791759,1463,Yale,A,1343,Princeton,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,1343 M. Allocco,1343 T. Evbuomwan,1343 K. Kellman,1343 R. Langborg,1343 C. Pierce,-0.473026
127636,2023,132,1,-0.693147,1336,Penn St,N,1345,Purdue,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,1345 Z. Edey,1345 M. Gillis,1345 F. Loyer,1345 B. Newman,1345 B. Smith,1.020525


Get player ratings

In [68]:
%%time

ratings = []

for season in df_reg_reshape['season'].unique():
    print(season)
    
    temp = pd.get_dummies(
        df_reg_reshape.loc[df_reg_reshape['season'] == season, 
                           list(df_reg_reshape.filter(regex='starter').columns)
                          ].reset_index(drop=True).copy()
    )
    
    temp.columns = temp.columns.str[:5] + temp.columns.str[14:]
    
    X_train = temp.groupby(temp.columns, axis=1).sum()
    
    y_train = df_reg_reshape.loc[df_reg_reshape['season'] == season, 
                                 'log_score_diff'
                                ].reset_index(drop=True).copy()
    
    
    mod = Ridge(fit_intercept=False)
    mod.fit(X_train, y_train)
    
    r = pd.DataFrame(
        {'Season': season, 'Player': X_train.columns, 'Rating': mod.coef_}
    )
    
    r = r.loc[r['Player'].str.contains('team0'), :].reset_index(drop=True)
    
    r['Player'] = r['Player'].str[6:]
    
    ratings.append(r)
    
df_ratings = pd.concat(
    ratings, ignore_index=True
).sort_values(
    ['Rating'], ascending=False, ignore_index=True
)

df_ratings

2011
2012
2013
2014
2015
2016
2017
2018
2019
2021
2022
2023
CPU times: total: 7min 12s
Wall time: 2min 12s


,Season,Player,Rating
0,2021,1137 J. Meeks,3.111078
1,2015,1458 F. Kaminsky,2.296355
2,2021,1207 C. Bile,2.283802
3,2013,1129 J. Elorriaga,2.283612
4,2023,1426 M. Young,2.189802
...,...,...,...
38175,2011,1438 A. Mitchell,-2.408783
38176,2023,1185 L. Geeter,-2.611773
38177,2013,1283 B. Marshall,-2.662344
38178,2021,1148 S. Ayangma,-2.697488


In [69]:
df_ratings['Team'] = df_ratings['Player'].str[:4].astype(int)

df_ratings

,Season,Player,Rating,Team
0,2021,1137 J. Meeks,3.111078,1137
1,2015,1458 F. Kaminsky,2.296355,1458
2,2021,1207 C. Bile,2.283802,1207
3,2013,1129 J. Elorriaga,2.283612,1129
4,2023,1426 M. Young,2.189802,1426
...,...,...,...,...
38175,2011,1438 A. Mitchell,-2.408783,1438
38176,2023,1185 L. Geeter,-2.611773,1185
38177,2013,1283 B. Marshall,-2.662344,1283
38178,2021,1148 S. Ayangma,-2.697488,1148


In [70]:
df_ratings['TeamRating'] = np.array(
    pd.merge(
        df_ratings, df_team_ratings, 
        how='left', 
        on=['Season', 'Team']
    )['Rating_y']
)

df_ratings

,Season,Player,Rating,Team,TeamRating
0,2021,1137 J. Meeks,3.111078,1137,-0.838011
1,2015,1458 F. Kaminsky,2.296355,1458,3.173469
2,2021,1207 C. Bile,2.283802,1207,1.002276
3,2013,1129 J. Elorriaga,2.283612,1129,1.623715
4,2023,1426 M. Young,2.189802,1426,-0.790281
...,...,...,...,...,...
38175,2011,1438 A. Mitchell,-2.408783,1438,0.720895
38176,2023,1185 L. Geeter,-2.611773,1185,-1.347487
38177,2013,1283 B. Marshall,-2.662344,1283,-0.425138
38178,2021,1148 S. Ayangma,-2.697488,1148,-1.806089


In [71]:
df_ratings['Rating'] = df_ratings['Rating'] + df_ratings['TeamRating']

df_ratings.sort_values(['Rating'], ascending=False, inplace=True, ignore_index=True)

df_ratings

,Season,Player,Rating,Team,TeamRating
0,2015,1458 F. Kaminsky,5.469824,1458,3.173469
1,2019,1181 C. Reddish,5.110943,1181,3.301051
2,2016,1242 L. Lucas,4.986190,1242,3.075190
3,2015,1246 A. Harrison,4.970060,1246,3.642318
4,2012,1246 M. Kidd-Gilchrist,4.939966,1246,3.188301
...,...,...,...,...,...
38175,2013,1212 P. Roberson,-4.768758,1212,-3.955867
38176,2015,1212 R. Freeman,-4.798835,1212,-3.445263
38177,2013,1212 D. Wheeler,-4.912330,1212,-3.955867
38178,2013,1212 S. Danridge,-5.409461,1212,-3.955867


Get last available starting lineups

In [72]:
df_last_starters = df_reg_reshape.groupby(
    ['season', 'team0']
)[['season', 'team0', 
   'team0_starter1', 
   'team0_starter2', 
   'team0_starter3', 
   'team0_starter4', 
   'team0_starter5'
  ]].tail(1).copy()

df_last_starters

,season,team0,team0_starter1,team0_starter2,team0_starter3,team0_starter4,team0_starter5
20687,2014,1455,1455 R. Baker,1455 K. Coleby,1455 T. Cotton,1455 C. Early,1455 F. VanVleet
26175,2015,1246,1246 W. Cauley-Stein,1246 A. Harrison,1246 A. Harrison,1246 T. Lyles,1246 K. Towns
52820,2021,1211,1211 J. Ayayi,1211 C. Kispert,1211 A. Nembhard,1211 J. Suggs,1211 D. Timme
66849,2011,1254,1254 J. Boyd,1254 C. Garner,1254 D. Hicks,1254 K. Johnson,1254 J. Olasewere
66930,2011,1125,1125 I. Clark,1125 D. Hanlen,1125 M. Hedgepeth,1125 J. House,1125 T. Noack
...,...,...,...,...,...,...,...
127633,2023,1401,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor
127634,2023,1222,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker
127635,2023,1463,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas
127636,2023,1336,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter


In [73]:
df_last_starters['starter1_rating'] = np.array(
    pd.merge(
        df_last_starters, df_ratings, how='left', 
        left_on=['season', 'team0_starter1'], 
        right_on=['Season', 'Player']
    )['Rating']
)

df_last_starters['starter2_rating'] = np.array(
    pd.merge(
        df_last_starters, df_ratings, how='left', 
        left_on=['season', 'team0_starter2'], 
        right_on=['Season', 'Player']
    )['Rating']
)

df_last_starters['starter3_rating'] = np.array(
    pd.merge(
        df_last_starters, df_ratings, how='left', 
        left_on=['season', 'team0_starter3'], 
        right_on=['Season', 'Player']
    )['Rating']
)

df_last_starters['starter4_rating'] = np.array(
    pd.merge(
        df_last_starters, df_ratings, how='left', 
        left_on=['season', 'team0_starter4'], 
        right_on=['Season', 'Player']
    )['Rating']
)

df_last_starters['starter5_rating'] = np.array(
    pd.merge(
        df_last_starters, df_ratings, how='left', 
        left_on=['season', 'team0_starter5'], 
        right_on=['Season', 'Player']
    )['Rating']
)

df_last_starters

,season,team0,team0_starter1,team0_starter2,team0_starter3,team0_starter4,team0_starter5,starter1_rating,starter2_rating,starter3_rating,starter4_rating,starter5_rating
20687,2014,1455,1455 R. Baker,1455 K. Coleby,1455 T. Cotton,1455 C. Early,1455 F. VanVleet,3.545423,3.320301,3.545423,3.545423,3.545423
26175,2015,1246,1246 W. Cauley-Stein,1246 A. Harrison,1246 A. Harrison,1246 T. Lyles,1246 K. Towns,3.941789,4.970060,4.970060,4.142049,4.670588
52820,2021,1211,1211 J. Ayayi,1211 C. Kispert,1211 A. Nembhard,1211 J. Suggs,1211 D. Timme,3.708037,4.240831,3.288199,4.603274,4.240831
66849,2011,1254,1254 J. Boyd,1254 C. Garner,1254 D. Hicks,1254 K. Johnson,1254 J. Olasewere,1.182577,1.182577,1.398360,1.182577,0.944960
66930,2011,1125,1125 I. Clark,1125 D. Hanlen,1125 M. Hedgepeth,1125 J. House,1125 T. Noack,2.212615,2.212615,2.212615,2.212615,1.759963
...,...,...,...,...,...,...,...,...,...,...,...,...
127633,2023,1401,1401 H. Coleman,1401 D. Dennis,1401 J. Marble,1401 T. Radford,1401 W. Taylor,2.672201,2.286053,3.274687,2.194877,1.650424
127634,2023,1222,1222 T. Arceneaux,1222 T. Mark,1222 J. Roberts,1222 J. Shead,1222 J. Walker,2.117733,3.426001,2.938312,3.426001,3.734324
127635,2023,1463,1463 E. Jarvis,1463 I. Kelly,1463 A. Mahoney,1463 B. Mbeng,1463 J. Poulakidas,0.672355,1.909842,0.764663,1.669998,1.669998
127636,2023,1336,1336 A. Funk,1336 S. Lundy,1336 K. Njie,1336 J. Pickett,1336 C. Wynter,1.970809,1.967993,1.231796,1.970809,2.711697


In [74]:
df = pd.read_pickle('../resources/data/preprocessed/tournament_games.pkl')

df = df.loc[df['season'] >= 2011, :].reset_index(drop=True).copy()

df

,winner,season,day_number,team0,team1,prev_matchups,common_opps_win_pct_diff,elo_diff,ordinal_elo_diff,BIH,...,PGH,POM,REW,RTH,SAG,STH,WIL,WLK,WOB,WOL
0,0,2011,134,1155,1412,NaN,0.000000,23.658444,-5,3.0,...,1.0,32.0,-12.0,20.0,11.0,15.0,4.0,13.0,-2.0,-12.0
1,0,2011,134,1421,1114,NaN,NaN,-58.217711,34,18.0,...,29.0,67.0,46.0,33.0,52.0,80.0,27.0,83.0,22.0,48.0
2,0,2011,135,1427,1106,NaN,NaN,-129.057955,61,85.0,...,72.0,76.0,50.0,90.0,89.0,87.0,92.0,67.0,77.0,77.0
3,0,2011,135,1433,1425,NaN,0.166667,-44.819201,16,2.0,...,8.0,-40.0,40.0,-12.0,-15.0,-19.0,3.0,4.0,8.0,30.0
4,0,2011,136,1139,1330,NaN,0.250000,139.996746,-28,-31.0,...,-28.0,-2.0,-29.0,-22.0,-7.0,-21.0,-32.0,-14.0,-32.0,-26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,1,2022,146,1274,1242,NaN,0.000000,239.449424,-45,-42.0,...,-43.0,-54.0,-39.0,-48.0,-61.0,-46.0,-42.0,-50.0,-45.0,-44.0
1468,1,2022,146,1389,1314,NaN,NaN,212.406858,-88,-102.0,...,-105.0,-88.0,-111.0,-111.0,-114.0,-106.0,-106.0,-83.0,-100.0,-93.0
1469,1,2022,152,1437,1242,NaN,-0.083333,7.104325,-1,-5.0,...,-2.0,-5.0,-1.0,-4.0,-6.0,-7.0,-2.0,-7.0,-3.0,-1.0
1470,1,2022,152,1181,1314,0.5,-0.094203,-85.575138,11,18.0,...,13.0,18.0,16.0,23.0,14.0,19.0,21.0,15.0,19.0,18.0


In [75]:
temp1 = pd.merge(
    df[['season', 'team1']], df_last_starters[['season', 'team0'] + [f'starter{i}_rating' 
                                                                     for i in range(1, 6)]
                                             ], how='left', 
    left_on=['season', 'team1'], right_on=['season', 'team0']
)

temp1['starters_rating'] = np.mean(temp1[[f'starter{i}_rating' for i in range(1, 6)]], axis=1)

temp1

,season,team1,team0,starter1_rating,starter2_rating,starter3_rating,starter4_rating,starter5_rating,starters_rating
0,2011,1412,1412,1.721657,2.433422,2.087397,2.087397,0.849718,1.835918
1,2011,1114,1114,-1.013576,-0.686009,-0.728585,-1.056151,-0.726423,-0.842149
2,2011,1106,1106,-0.713566,-0.944602,-1.369958,-1.669348,-1.129193,-1.165333
3,2011,1425,1425,2.176601,2.060395,2.512853,2.060395,2.060395,2.174128
4,2011,1330,1330,2.375257,1.740178,1.999903,1.999903,1.959909,2.015030
...,...,...,...,...,...,...,...,...,...
1467,2022,1242,1242,3.234418,3.618314,3.618314,3.809953,3.175025,3.491205
1468,2022,1314,1314,2.071210,2.548206,2.071210,2.419976,1.462073,2.114535
1469,2022,1242,1242,3.234418,3.618314,3.618314,3.809953,3.175025,3.491205
1470,2022,1314,1314,2.071210,2.548206,2.071210,2.419976,1.462073,2.114535


In [76]:
temp0 = pd.merge(
    df[['season', 'team0']], df_last_starters[['season', 'team0'] + [f'starter{i}_rating' 
                                                                     for i in range(1, 6)]
                                             ], how='left', 
    on=['season', 'team0']
)

temp0['starters_rating'] = np.mean(temp0[[f'starter{i}_rating' for i in range(1, 6)]], axis=1)

temp0

,season,team0,starter1_rating,starter2_rating,starter3_rating,starter4_rating,starter5_rating,starters_rating
0,2011,1155,2.818812,2.313616,1.934958,2.467856,2.000122,2.307073
1,2011,1421,0.211228,0.115362,0.211228,0.211228,0.370322,0.223874
2,2011,1427,-0.406765,-0.406765,-0.406765,0.751930,-1.146768,-0.323027
3,2011,1433,1.297087,1.733037,1.835636,1.297087,1.408978,1.514365
4,2011,1139,1.056771,2.226401,1.438644,2.040553,1.884303,1.729335
...,...,...,...,...,...,...,...,...
1467,2022,1274,1.834734,1.795903,1.834734,1.475329,1.747162,1.737572
1468,2022,1389,0.081379,0.369710,1.220566,1.295658,0.801252,0.753713
1469,2022,1437,3.247558,3.247558,2.429840,3.602939,3.247558,3.155091
1470,2022,1181,3.003321,2.749508,2.406873,3.003321,3.003321,2.833269


In [78]:
df['starters_diff'] = temp1['starters_rating'] - temp0['starters_rating']

df

,winner,season,day_number,team0,team1,prev_matchups,common_opps_win_pct_diff,elo_diff,ordinal_elo_diff,BIH,...,POM,REW,RTH,SAG,STH,WIL,WLK,WOB,WOL,starters_diff
0,0,2011,134,1155,1412,NaN,0.000000,23.658444,-5,3.0,...,32.0,-12.0,20.0,11.0,15.0,4.0,13.0,-2.0,-12.0,-0.471155
1,0,2011,134,1421,1114,NaN,NaN,-58.217711,34,18.0,...,67.0,46.0,33.0,52.0,80.0,27.0,83.0,22.0,48.0,-1.066023
2,0,2011,135,1427,1106,NaN,NaN,-129.057955,61,85.0,...,76.0,50.0,90.0,89.0,87.0,92.0,67.0,77.0,77.0,-0.842307
3,0,2011,135,1433,1425,NaN,0.166667,-44.819201,16,2.0,...,-40.0,40.0,-12.0,-15.0,-19.0,3.0,4.0,8.0,30.0,0.659763
4,0,2011,136,1139,1330,NaN,0.250000,139.996746,-28,-31.0,...,-2.0,-29.0,-22.0,-7.0,-21.0,-32.0,-14.0,-32.0,-26.0,0.285695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,1,2022,146,1274,1242,NaN,0.000000,239.449424,-45,-42.0,...,-54.0,-39.0,-48.0,-61.0,-46.0,-42.0,-50.0,-45.0,-44.0,1.753633
1468,1,2022,146,1389,1314,NaN,NaN,212.406858,-88,-102.0,...,-88.0,-111.0,-111.0,-114.0,-106.0,-106.0,-83.0,-100.0,-93.0,1.360822
1469,1,2022,152,1437,1242,NaN,-0.083333,7.104325,-1,-5.0,...,-5.0,-1.0,-4.0,-6.0,-7.0,-2.0,-7.0,-3.0,-1.0,0.336114
1470,1,2022,152,1181,1314,0.5,-0.094203,-85.575138,11,18.0,...,18.0,16.0,23.0,14.0,19.0,21.0,15.0,19.0,18.0,-0.718734


In [79]:
temp1 = np.array(
    pd.merge(
        df, df_team_ratings, how='left', 
        left_on=['season', 'team1'], 
        right_on=['Season', 'Team']
    )['Rating']
)

temp0 = np.array(
    pd.merge(
        df, df_team_ratings, how='left', 
        left_on=['season', 'team0'], 
        right_on=['Season', 'Team']
    )['Rating']
)

df['rating_diff'] = temp1 - temp0

df

,winner,season,day_number,team0,team1,prev_matchups,common_opps_win_pct_diff,elo_diff,ordinal_elo_diff,BIH,...,REW,RTH,SAG,STH,WIL,WLK,WOB,WOL,starters_diff,rating_diff
0,0,2011,134,1155,1412,NaN,0.000000,23.658444,-5,3.0,...,-12.0,20.0,11.0,15.0,4.0,13.0,-2.0,-12.0,-0.471155,-0.439765
1,0,2011,134,1421,1114,NaN,NaN,-58.217711,34,18.0,...,46.0,33.0,52.0,80.0,27.0,83.0,22.0,48.0,-1.066023,-0.768886
2,0,2011,135,1427,1106,NaN,NaN,-129.057955,61,85.0,...,50.0,90.0,89.0,87.0,92.0,67.0,77.0,77.0,-0.842307,-0.872944
3,0,2011,135,1433,1425,NaN,0.166667,-44.819201,16,2.0,...,40.0,-12.0,-15.0,-19.0,3.0,4.0,8.0,30.0,0.659763,0.436562
4,0,2011,136,1139,1330,NaN,0.250000,139.996746,-28,-31.0,...,-29.0,-22.0,-7.0,-21.0,-32.0,-14.0,-32.0,-26.0,0.285695,-0.013491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,1,2022,146,1274,1242,NaN,0.000000,239.449424,-45,-42.0,...,-39.0,-48.0,-61.0,-46.0,-42.0,-50.0,-45.0,-44.0,1.753633,1.419099
1468,1,2022,146,1389,1314,NaN,NaN,212.406858,-88,-102.0,...,-111.0,-111.0,-114.0,-106.0,-106.0,-83.0,-100.0,-93.0,1.360822,1.254796
1469,1,2022,152,1437,1242,NaN,-0.083333,7.104325,-1,-5.0,...,-1.0,-4.0,-6.0,-7.0,-2.0,-7.0,-3.0,-1.0,0.336114,0.252394
1470,1,2022,152,1181,1314,0.5,-0.094203,-85.575138,11,18.0,...,16.0,23.0,14.0,19.0,21.0,15.0,19.0,18.0,-0.718734,-0.636140


In [217]:
df.to_pickle('../resources/data/preprocessed/tournament_games_with_starters.pkl')

'Done'

'Done'